In [10]:
import scipy.io.wavfile
import numpy as np 
from numpy.fft import fft, ifft 

Fs, audio = scipy.io.wavfile.read('audio2.wav')
arr_data = np.genfromtxt('../../datasets/vocales.csv', delimiter=',')
arr_labels = np.genfromtxt('../../datasets/vocales.csv', delimiter=',', dtype=str)[0:,-1]


#elimino cabecera y output
arr_data = np.delete(arr_data, [-1], 1)

In [11]:
def transform_fourier(cut):
  fourier = fft(cut)
  threeKHzCut = int(3000*len(cut)/Fs)
  return fourier[0: threeKHzCut], fourier.size

def delete_noise_from_points(Fsmall, cut):
  filter = int(125*len(cut)/Fs)
  to_process = np.sqrt((np.real(Fsmall)**2+np.imag(Fsmall)**2))
  out = np.zeros(len(to_process)-filter, dtype=np.float64)

  for i in range(len(to_process)-filter):
      for j in range(filter):
          out[i] += to_process[i+j]

  return out

def get_max_points(cut, out):
  filter = int(125*len(cut)/Fs)
  cut_filter = int(250 * len(cut) / Fs)
  max_points = []


  for i in range(3):
    curr_max_point = np.argmax(out) + filter
    max_points.append(curr_max_point)

    start_cut = curr_max_point-filter-cut_filter
    end_cut = curr_max_point + cut_filter - filter
    if (start_cut < 0):
      start_cut = 0
  
    out[start_cut:end_cut] = 0
  
  max_points.sort()
  
  return max_points

  
   
  

def create_vocals_dataset(audio, Fs, arr_data, arr_labels):
  output = []
  for i in range(len(arr_data)):
  
    start, end = arr_data[i]
    cut_label = arr_labels[i]

    cut = audio[int(start*Fs):int(end*Fs)]
    Fsmall, fourier_size = transform_fourier(cut)
    out = delete_noise_from_points(Fsmall, cut)
    max_points = get_max_points(cut, out)
    max_points = [float("{:.2f}".format(i*Fs/fourier_size)) for i in max_points ]

    output.append(max_points + [cut_label])
  

  np.savetxt('./../../datasets/vocals-dataset.csv', output, fmt='%s,%s,%s,%s')

In [12]:

create_vocals_dataset(audio, Fs, arr_data, arr_labels)